In [1]:
import reservoirflow as rf
import numpy as np

In [2]:
rf.__version__

'0.0.1'

In [3]:
type(rf.scalers.MinMax((1, 1)))

reservoirflow.scalers.minmax.MinMax

In [ ]:
def create_model_example_7_1():
    grid = rf.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rf.models.Numerical(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rf.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rf.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rf.models.Numerical(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

In [ ]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

In [ ]:
np.count_nonzero(np.array([1, 2, 0, 0]) == 0)

In [ ]:
scaler = rf.scalers.MinMax((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

In [ ]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

In [ ]:
t

In [ ]:
t, x = model.get_domain(False, True)
t, x

In [ ]:
rf.scalers.MinMax((-1, 1)).fit_transform(x, axis=0)

In [ ]:
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

In [ ]:
model.get_centers(False)

In [ ]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [ ]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

In [ ]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

In [ ]:
model.get_df(
    scale=True,
    melt=False,
    boundary=True,
    units=True,
    columns=["time", "cells", "wells"],
)